## Deliverable 2. Create a Customer Travel Destinations Map.

In [26]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np 

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Description,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Allapalli,IN,broken clouds,19.4167,80.0667,64.54,28,60,3.91
1,1,Marinette,US,clear sky,45.1000,-87.6307,-5.80,71,1,9.22
2,2,New Norfolk,AU,overcast clouds,-42.7826,147.0587,55.40,58,90,9.22
3,3,Jamestown,US,light snow,42.0970,-79.2353,21.00,86,90,9.22
4,4,Klaksvik,FO,scattered clouds,62.2266,-6.5890,33.80,86,40,5.75


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Description,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
8,8,Karema,TZ,broken clouds,-6.8205,30.4389,76.30,76,83,4.52
9,9,Vaini,TO,moderate rain,-21.2000,-175.2000,77.00,94,75,10.36
12,12,Rikitea,PF,overcast clouds,-23.1203,-134.9692,77.27,72,95,11.77
16,16,Kununurra,AU,scattered clouds,-15.7667,128.7333,78.80,100,40,2.30
21,21,Gigmoto,PH,overcast clouds,13.7796,124.3900,78.87,84,99,15.39
24,24,Sarangani,PH,overcast clouds,5.4033,125.4636,81.79,76,100,4.76
28,28,Tirumullaivasal,IN,scattered clouds,11.2333,79.8333,75.94,71,35,13.67
31,31,Arraial Do Cabo,BR,moderate rain,-22.9661,-42.0278,77.00,83,75,9.22
32,32,Santa Maria,BR,few clouds,-29.6842,-53.8069,84.20,51,20,6.91
38,38,Puerto Ayora,EC,clear sky,-0.7393,-90.3518,84.20,70,0,18.41


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID        225
City           225
Country        221
Description    225
Lat            225
Lng            225
Max Temp       225
Humidity       225
Cloudiness     225
Wind Speed     225
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
vacation_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)].dropna()
vacation_cities_df.head(10)

,City_ID,City,Country,Description,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
8,8,Karema,TZ,broken clouds,-6.8205,30.4389,76.30,76,83,4.52
9,9,Vaini,TO,moderate rain,-21.2000,-175.2000,77.00,94,75,10.36
12,12,Rikitea,PF,overcast clouds,-23.1203,-134.9692,77.27,72,95,11.77
16,16,Kununurra,AU,scattered clouds,-15.7667,128.7333,78.80,100,40,2.30
21,21,Gigmoto,PH,overcast clouds,13.7796,124.3900,78.87,84,99,15.39
24,24,Sarangani,PH,overcast clouds,5.4033,125.4636,81.79,76,100,4.76
28,28,Tirumullaivasal,IN,scattered clouds,11.2333,79.8333,75.94,71,35,13.67
31,31,Arraial Do Cabo,BR,moderate rain,-22.9661,-42.0278,77.00,83,75,9.22
32,32,Santa Maria,BR,few clouds,-29.6842,-53.8069,84.20,51,20,6.91
38,38,Puerto Ayora,EC,clear sky,-0.7393,-90.3518,84.20,70,0,18.41


In [28]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = vacation_cities_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = np.nan
hotel_df.head()

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
8,Karema,TZ,76.30,broken clouds,-6.8205,30.4389,NaN
9,Vaini,TO,77.00,moderate rain,-21.2000,-175.2000,NaN
12,Rikitea,PF,77.27,overcast clouds,-23.1203,-134.9692,NaN
16,Kununurra,AU,78.80,scattered clouds,-15.7667,128.7333,NaN
21,Gigmoto,PH,78.87,overcast clouds,13.7796,124.3900,NaN


In [30]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [34]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
9,Vaini,TO,77.00,moderate rain,-21.2000,-175.2000,Keleti Beach Resort
12,Rikitea,PF,77.27,overcast clouds,-23.1203,-134.9692,Pension Maro'i
16,Kununurra,AU,78.80,scattered clouds,-15.7667,128.7333,Hotel Kununurra
21,Gigmoto,PH,78.87,overcast clouds,13.7796,124.3900,Cj's Travellers Inn
24,Sarangani,PH,81.79,overcast clouds,5.4033,125.4636,Ballistic Islet
...,...,...,...,...,...,...,...
710,La Libertad,EC,82.40,broken clouds,-2.2333,-80.9000,Playa Canela Hotel Boutique
722,Trinidad,BO,87.80,broken clouds,-14.8333,-64.9000,Hotel Piesta
729,Isabela,PH,78.80,broken clouds,6.7085,121.9711,Isawad Resort
736,Grand-Lahou,CI,83.50,broken clouds,5.1367,-5.0242,Hotel Beatitude de Grand Lahou


In [35]:
# 8a. Create the output File (CSV)
output_data_file = "hotel_database"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [36]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [40]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))